### Imports

In [1]:
from tqdm import tqdm
import random
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import gymnasium as gym
import gym_mtsim
sys.path.append("C:/Users/WilliamFetzner/Documents/Trading/")
from gym_mtsim_forked.gym_mtsim.data import FOREX_DATA_PATH_TRAIN, FOREX_DATA_PATH_TEST, FOREX_DATA_PATH
from gym_mtsim import OrderType, Timeframe, MtEnv, MtSimulator
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials, STATUS_FAIL
from stable_baselines3 import A2C, PPO
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
import time
import torch

c:\Users\WilliamFetzner\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment forex-hedge-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
c:\Users\WilliamFetzner\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment forex-unhedge-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
c:\Users\WilliamFetzner\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment stocks-hedge-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
c:\Users\WilliamFetzner\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment stocks-unhedge-v0 alr

# Parameters

In [2]:
# import pickle
# import time
# from hyperopt import fmin, tpe, hp, STATUS_OK, Trials


# def objective(x):
#     return {
#         'loss': x ** 2,
#         'status': STATUS_OK,
#         # -- store other results like this
#         'eval_time': time.time(),
#         'other_stuff': {'type': None, 'value': x},
#         # -- attachments are handled differently
#         'attachments':
#             {'time_module': pickle.dumps(time.time)}
#         }
# trials = Trials()
# best = fmin(objective,
#             space=hp.uniform('x', -10, 10),
#             algo=tpe.suggest,
#             max_evals=100,
#             trials=trials)

# print(best)

In [3]:
# trials.results

In [4]:
import pickle
# unpack the pickle file and load the data that is in symbols_forex.pkl
with open('C:/Users/WilliamFetzner/Documents/Trading/gym_mtsim_forked/gym_mtsim/data/symbols_forex.pkl', 'rb') as f:
    symbols = pickle.load(f)
# convert symbols to a pd.dataframe
# symbols[1]['EURUSD']
split = int(len(symbols[1]['EURUSD']) * 0.80)
validation_split = int(len(symbols[1]['EURUSD']) * 0.90)

In [5]:
# get the 2 weeks of the symbols[1]['EURUSD'] dataframe by first finding the max date
# then subtracting 14 days from that date
symbols[1]['EURUSD'].index = pd.to_datetime(symbols[1]['EURUSD'].index)
max_date = symbols[1]['EURUSD'].index.max()

In [6]:
# find the previous friday before max_date
# what is the day of the week of the max_date
max_day_of_week = max_date.dayofweek
# subtract the day of the week from the max_date to get the previous friday
max_friday = max_date - pd.DateOffset(days=max_day_of_week+2)
two_weeks = max_friday - pd.DateOffset(days=14)
one_week = max_friday - pd.DateOffset(days=7)

In [7]:
training_index_slice = symbols[1]['EURUSD'].loc[:two_weeks, :].index
validation_index_slice = symbols[1]['EURUSD'].loc[two_weeks:one_week, :].index
testing_index_slice = symbols[1]['EURUSD'].loc[one_week:, :].index

In [8]:
validation_index_slice

DatetimeIndex(['2024-04-01 00:00:00+00:00', '2024-04-01 01:00:00+00:00',
               '2024-04-01 02:00:00+00:00', '2024-04-01 03:00:00+00:00',
               '2024-04-01 04:00:00+00:00', '2024-04-01 05:00:00+00:00',
               '2024-04-01 06:00:00+00:00', '2024-04-01 07:00:00+00:00',
               '2024-04-01 08:00:00+00:00', '2024-04-01 09:00:00+00:00',
               ...
               '2024-04-05 14:00:00+00:00', '2024-04-05 15:00:00+00:00',
               '2024-04-05 16:00:00+00:00', '2024-04-05 17:00:00+00:00',
               '2024-04-05 18:00:00+00:00', '2024-04-05 19:00:00+00:00',
               '2024-04-05 20:00:00+00:00', '2024-04-05 21:00:00+00:00',
               '2024-04-05 22:00:00+00:00', '2024-04-05 23:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Time', length=120, freq=None)

### Create Env

In [9]:
sim_train = gym_mtsim.MtSimulator(
    unit='USD',
    balance=200000.,
    leverage=100.,
    stop_out_level=0.2,
    hedge=True,
    symbols_filename=FOREX_DATA_PATH
)

env_train = MtEnv(
    original_simulator=sim_train,
    trading_symbols=['EURUSD'],
    window_size = 10,
    time_points=list(training_index_slice),
    hold_threshold=0.5,
    close_threshold=0.5,
    fee=lambda symbol: {
        # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
        'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
        # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
    }[symbol],
    symbol_max_orders=2,
    multiprocessing_processes=2
)

In [10]:
sim_validation = gym_mtsim.MtSimulator(
    unit='USD',
    balance=200000.,
    leverage=100.,
    stop_out_level=0.2,
    hedge=True,
    symbols_filename=FOREX_DATA_PATH
)

env_validation = MtEnv(
    original_simulator=sim_validation,
    trading_symbols=['EURUSD'],
    window_size = 10,
    time_points=list(validation_index_slice),
    hold_threshold=0.5,
    close_threshold=0.5,
    fee=lambda symbol: {
        # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
        'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
        # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
    }[symbol],
    symbol_max_orders=2,
    multiprocessing_processes=2
)

In [11]:
sim_testing = gym_mtsim.MtSimulator(
    unit='USD',
    balance=200000.,
    leverage=100.,
    stop_out_level=0.2,
    hedge=True,
    symbols_filename=FOREX_DATA_PATH
)

env_testing = MtEnv(
    original_simulator=sim_testing,
    trading_symbols=['EURUSD'],
    window_size = 10,
    time_points=list(testing_index_slice),
    hold_threshold=0.5,
    close_threshold=0.5,
    fee=lambda symbol: {
        # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
        'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
        # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
    }[symbol],
    symbol_max_orders=2,
    multiprocessing_processes=2
)

### Define Functions

In [12]:
def print_stats(reward_over_episodes, printing_name):
    """  Print Reward  """

    avg_rewards = np.mean(reward_over_episodes)
    min_rewards = np.min(reward_over_episodes)
    max_rewards = np.max(reward_over_episodes)

    print (f'Min. {printing_name}          : {min_rewards:>10.3f}')
    print (f'Avg. {printing_name}          : {avg_rewards:>10.3f}')
    print (f'Max. {printing_name}          : {max_rewards:>10.3f}')

    return min_rewards, avg_rewards, max_rewards


# ProgressBarCallback for model.learn()
class ProgressBarCallback(BaseCallback):

    def __init__(self, check_freq: int, verbose: int = 1):
        super().__init__(verbose)
        self.check_freq = check_freq

    def _on_training_start(self) -> None:
        """
        This method is called before the first rollout starts.
        """
        self.progress_bar = tqdm(total=self.model._total_timesteps, desc="model.learn()")

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:
            self.progress_bar.update(self.check_freq)
        return True
    
    def _on_training_end(self) -> None:
        """
        This event is triggered before exiting the `learn()` method.
        """
        self.progress_bar.close()


In [13]:
space = {
    'learning_rate': hp.loguniform('learning_rate', -5, -2), # Learning rate
    # 'gamma': hp.uniform('gamma', 0.97, 0.99), # Discount factor
    'ent_coef': hp.loguniform('ent_coef', -5, 0), # Entropy coefficient
    # 'learning_timesteps': hp.choice('learning_timesteps', [25, 50, 100, 250, 500]),
    # 'window_size': hp.choice('window_size', [10, 20, 50]) # Window size
}

In [14]:
# TRAINING + TEST
def train_val_model(model, model_policy, env_tr, env_val, seed, steps_str, lr, gamma_param, entropy, total_learning_timesteps=10_000):
    """
    Trains and validates a model using the Proximal Policy Optimization (PPO) algorithm.

    Args:
        model (object): The model to be trained.
        model_policy (object): The policy used by the model.
        env_tr (object): The training environment.
        env_val (object): The validation environment.
        seed (int): The random seed for reproducibility.
        steps_str (str): A string representing the number of steps.
        window_size_param (int): The window size parameter.
        lr (float): The learning rate.
        gamma_param (float): The gamma parameter.
        entropy (float): The entropy coefficient.
        total_learning_timesteps (int, optional): The total number of learning timesteps. Defaults to 10,000.

    Returns:
        tuple: A tuple containing the reward over validations, orders over validations, and the model dictionary.
    """
    # reproduce training and test
    print('-' * 80)
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    #model_dict = {}
    # env_tr.window_size = window_size_param
    print(f'entropy: {entropy}, learning rate: {lr}')
    # eval_callback = EvalCallback(env_tr, log_path='./logs/', eval_freq=1000)
    model = PPO(model_policy, env_tr, verbose=0, ent_coef=entropy, learning_rate=lr)#, gamma=gamma_param, 
    obs_tr, info_tr = env_tr.reset(seed=seed)
    # custom callback for 'progress_bar'
    model.learn(total_timesteps=total_learning_timesteps)#, callback=ProgressBarCallback(100))

    reward_over_validations = []
    orders_over_validations = []

    for episode in range(0, 10):
        obs_val, info_val = env_val.reset(seed=seed)

        total_reward = 0
        done_val = False

        while not done_val:
            action, _states = model.predict(obs_val)
            obs_val, reward_val, terminated_val, truncated_val, info_val = env_val.step(action)
            done_val = terminated_val or truncated_val

            total_reward += reward_val
            if done_val:
                break
        try:
            order_len = len(env_val.render()['orders'])
        except:
            order_len = 0

        # model_dict[f'model_{episode}'] = model
        # model.save(f'models_4_19_24/window_{window_size_param}_entropy_{round(entropy, 4)}/model_{steps_str}_{episode}.pkl')

        reward_over_validations.append(total_reward)    
        orders_over_validations.append(order_len)  


        # if episode % 1 == 0:
        avg_reward = np.mean(reward_over_validations)
        avg_orders = np.mean(orders_over_validations)
        print(f'Episode: {episode}, Avg. Reward: {avg_reward:.3f}, # of orders: {avg_orders:.3f}')

    return reward_over_validations, orders_over_validations#, model_dict

### Train + Test Env

In [15]:
seed = 2024  # random seed
total_num_episodes = 10

# print ("env_name                 :", env_name)
print ("seed                     :", seed)

# INIT matplotlib
plot_settings = {}
plot_data = {'x': [i for i in range(1, total_num_episodes + 1)]}

# learning_timesteps_list_in_K = [25]#, 50, 100]
# learning_timesteps_list_in_K = [50, 250, 500]
# learning_timesteps_list_in_K = [500, 1000, 3000, 5000]

# RL Algorithms: https://stable-baselines3.readthedocs.io/en/master/guide/algos.html

timesteps_models_dict = {}
def objective(params):
    learning_timesteps = 50 #params['learning_timesteps']
    ent_coef = params['ent_coef']
    gamma = 0.99 #params['gamma']
    learning_rate = params['learning_rate']#0.0003#

    if learning_rate > 0.05:
        print(f'Learning rate too high: {learning_rate}')
        return {'loss': None, 'status': STATUS_FAIL, 'eval_time': time.time(), 'parameters': params}
    if ent_coef > 0.1:
        print(f'Entropy too high: {ent_coef}')
        return {'loss': None, 'status': STATUS_FAIL, 'eval_time': time.time(), 'parameters': params}

    total_learning_timesteps = learning_timesteps * 1000
    step_key = f'{learning_timesteps}K'
    policy_dict = PPO.policy_aliases
    policy = policy_dict.get('MultiInputPolicy')
    class_name = type(PPO).__qualname__
    plot_key = f'{class_name}_rewards_'+step_key
    try:
        print(f'length of training env time points: {len(env_train.time_points)}, \
              length of validation env time points: {len(env_validation.time_points)}')
        rewards, orders = train_val_model(PPO, policy, env_train, env_validation, seed, step_key,  
                                                    learning_rate, gamma, ent_coef, total_learning_timesteps)
    except:
        print(f'''there was an error with those parameters: timesteps: {learning_timesteps}, \n
              ent_coef: {ent_coef}, gamma: {gamma}, learning_rate: {learning_rate}''')
        return {'loss': None, 'status': STATUS_FAIL, 'eval_time': time.time(), 'parameters': params}
    # timesteps_models_dict[step_key] = models_dict
    min_rewards, avg_rewards, max_rewards, = print_stats(rewards, 'Reward')
    print_stats(orders, 'Orders')
    label = f'Avg. {avg_rewards:>7.2f} : {class_name} - {step_key}'
    plot_data[plot_key] = rewards
    plot_settings[plot_key] = {'label': label}
    params['avg_orders'] = np.mean(orders)       

    return {'loss': -avg_rewards, 'status': STATUS_OK, 'eval_time': time.time(), 'parameters': params} 

seed                     : 2024


In [16]:
# # check if it is working:
parameters = {
    # 'window_size': 10,
    # 'learning_timesteps': 25,
    'ent_coef': 0.008841807731982131,
    # 'gamma': 0.9484679718228304,
    'learning_rate': 0.021173768344759137
}
objective(parameters)

length of training env time points: 88053,               length of validation env time points: 120
--------------------------------------------------------------------------------
entropy: 0.008841807731982131, learning rate: 0.021173768344759137


Episode: 0, Avg. Reward: 40601.199, # of orders: 3.000
Episode: 1, Avg. Reward: 40209.028, # of orders: 3.000
Episode: 2, Avg. Reward: 32845.796, # of orders: 3.667
Episode: 3, Avg. Reward: 37816.695, # of orders: 3.250
Episode: 4, Avg. Reward: 38993.440, # of orders: 3.600
Episode: 5, Avg. Reward: 37907.031, # of orders: 3.333
Episode: 6, Avg. Reward: 38488.420, # of orders: 3.143
Episode: 7, Avg. Reward: 37549.309, # of orders: 3.000
Episode: 8, Avg. Reward: 35971.379, # of orders: 3.000
Episode: 9, Avg. Reward: 35137.533, # of orders: 3.000
Min. Reward          :  18119.333
Avg. Reward          :  35137.533
Max. Reward          :  52729.391
Min. Orders          :      2.000
Avg. Orders          :      3.000
Max. Orders          :      5.000


{'loss': -35137.53328972687,
 'status': 'ok',
 'eval_time': 1713819844.4422543,
 'parameters': {'ent_coef': 0.008841807731982131,
  'learning_rate': 0.021173768344759137,
  'avg_orders': 3.0}}

In [16]:

# PPO('MultiInputPolicy', env_train, verbose=0, ent_coef=parameters['ent_coef']).learn(total_timesteps=25_000) #, learning_rate=parameters['learning_rate'], gamma=parameters['gamma'], ent_coef=parameters['ent_coef']

In [17]:
for i in range(0, 35, 7):
    training_index_slice = symbols[1]['EURUSD'].loc[:(max_friday - pd.DateOffset(days=i+7)), :].index
    validation_index_slice = symbols[1]['EURUSD'].loc[(max_friday - pd.DateOffset(days=i+7)):(max_friday - pd.DateOffset(days=i)), :].index
    env_train.time_points = list(training_index_slice)
    env_validation.time_points = list(validation_index_slice)
    trials = Trials()
    best = fmin(fn=objective,
                space=space,
                algo=tpe.suggest,
                max_evals=50, # Number of evaluations of the objective function
                trials=trials,
                trials_save_file=f'hyperopt/trials_4_22_iter_{i}.pkl')

    print("Best parameters:", best)


--------------------------------------------------------------------------------
Window size: 10, entropy: 0.4889833258316074, learning rate: 0.05217454370277322
  0%|          | 0/25 [00:00<?, ?trial/s, best loss=?]

there was an error with those parameters: Window: 10, timesteps: 50, 

              ent_coef: 0.4889833258316074, gamma: 0.99, learning_rate: 0.05217454370277322
--------------------------------------------------------------------------------
Window size: 20, entropy: 0.012732824987492135, learning rate: 0.017710320445299122
there was an error with those parameters: Window: 20, timesteps: 50, 

              ent_coef: 0.012732824987492135, gamma: 0.99, learning_rate: 0.017710320445299122
--------------------------------------------------------------------------------
Window size: 20, entropy: 0.9002325972040649, learning rate: 0.00793247674022005
there was an error with those parameters: Window: 20, timesteps: 50, 

              ent_coef: 0.9002325972040649, gamma: 0.99, learning_rate: 0.00793247674022005
--------------------------------------------------------------------------------
Window size: 20, entropy: 0.07364014439317697, learning rate: 0.078593007472276
there was an error w

  0%|          | 0/10 [00:00<?, ?it/s]
Episode: 0, Avg. Reward: 151356.009, # of orders: 48.000:   0%|          | 0/10 [00:02<?, ?it/s]
Episode: 0, Avg. Reward: 151356.009, # of orders: 48.000:  10%|#         | 1/10 [00:02<00:20,  2.27s/it]
Episode: 1, Avg. Reward: 20095.045, # of orders: 25.500:  20%|##        | 2/10 [00:04<00:18,  2.27s/it] 
Episode: 1, Avg. Reward: 20095.045, # of orders: 25.500:  30%|###       | 3/10 [00:04<00:10,  1.50s/it]
Episode: 2, Avg. Reward: 63245.987, # of orders: 37.333:  40%|####      | 4/10 [00:07<00:09,  1.50s/it]
Episode: 2, Avg. Reward: 63245.987, # of orders: 37.333:  50%|#####     | 5/10 [00:07<00:06,  1.34s/it]
Episode: 3, Avg. Reward: 22648.953, # of orders: 38.500:  50%|#####     | 5/10 [00:09<00:06,  1.34s/it]
Episode: 3, Avg. Reward: 22648.953, # of orders: 38.500:  60%|######    | 6/10 [00:09<00:06,  1.65s/it]
Episode: 4, Avg. Reward: -3408.575, # of orders: 31.400:  70%|#######   | 7/10 [00:12<00:04,  1.65s/it]
Episode: 4, Avg. Reward: -3408

Min. Reward          : -120904.603                               
Avg. Reward          : -12731.252                                
Max. Reward          : 151356.009                                
Min. Orders          :      2.000                                
Avg. Orders          :     32.200                                
Max. Orders          :     61.000                                
--------------------------------------------------------------------------------   
Window size: 10, entropy: 0.01194302900502865, learning rate: 0.04020552432891841  
 20%|██        | 5/25 [16:55<1:11:50, 215.53s/trial, best loss: 12731.251831984668]

  0%|          | 0/10 [00:00<?, ?it/s]
Episode: 0, Avg. Reward: 171517.387, # of orders: 47.000:   0%|          | 0/10 [00:02<?, ?it/s]
Episode: 0, Avg. Reward: 171517.387, # of orders: 47.000:  10%|#         | 1/10 [00:02<00:23,  2.56s/it]
Episode: 1, Avg. Reward: 136098.092, # of orders: 48.500:  20%|##        | 2/10 [00:05<00:20,  2.56s/it]
Episode: 1, Avg. Reward: 136098.092, # of orders: 48.500:  30%|###       | 3/10 [00:05<00:11,  1.69s/it]
Episode: 2, Avg. Reward: 160974.662, # of orders: 52.667:  40%|####      | 4/10 [00:07<00:10,  1.69s/it]
Episode: 2, Avg. Reward: 160974.662, # of orders: 52.667:  50%|#####     | 5/10 [00:07<00:07,  1.47s/it]
Episode: 3, Avg. Reward: 129050.318, # of orders: 52.250:  50%|#####     | 5/10 [00:10<00:07,  1.47s/it]
Episode: 3, Avg. Reward: 129050.318, # of orders: 52.250:  60%|######    | 6/10 [00:10<00:06,  1.75s/it]
Episode: 4, Avg. Reward: 83549.335, # of orders: 46.800:  70%|#######   | 7/10 [00:12<00:05,  1.75s/it] 
Episode: 4, Avg. Reward:

...

Min. Reward          : -79428.671                                                  
Avg. Reward          : -34440.459                                                  
Max. Reward          :  24592.070                                                  
Min. Orders          :     46.000                                                  
Avg. Orders          :     50.800                                                  
Max. Orders          :     63.000                                                  
--------------------------------------------------------------------------------   
Window size: 10, entropy: 0.061648885205386854, learning rate: 0.009128266453103316
 92%|█████████▏| 23/25 [1:17:35<09:01, 270.81s/trial, best loss: 34440.45898532432]

  0%|          | 0/10 [00:00<?, ?it/s]
Episode: 0, Avg. Reward: -35653.075, # of orders: 35.000:   0%|          | 0/10 [00:02<?, ?it/s]
Episode: 0, Avg. Reward: -35653.075, # of orders: 35.000:  10%|#         | 1/10 [00:02<00:20,  2.28s/it]
Episode: 1, Avg. Reward: -65397.198, # of orders: 30.000:  20%|##        | 2/10 [00:04<00:18,  2.28s/it]
Episode: 1, Avg. Reward: -65397.198, # of orders: 30.000:  30%|###       | 3/10 [00:04<00:09,  1.41s/it]
Episode: 2, Avg. Reward: -67476.765, # of orders: 34.000:  40%|####      | 4/10 [00:06<00:08,  1.41s/it]
Episode: 2, Avg. Reward: -67476.765, # of orders: 34.000:  50%|#####     | 5/10 [00:06<00:06,  1.24s/it]
Episode: 3, Avg. Reward: -60125.631, # of orders: 35.500:  50%|#####     | 5/10 [00:08<00:06,  1.24s/it]
Episode: 3, Avg. Reward: -60125.631, # of orders: 35.500:  60%|######    | 6/10 [00:08<00:05,  1.50s/it]
Episode: 4, Avg. Reward: -49322.214, # of orders: 36.200:  70%|#######   | 7/10 [00:11<00:04,  1.50s/it]
Episode: 4, Avg. Reward:

Min. Reward          : -99174.879                                                  
Avg. Reward          : -58715.615                                                  
Max. Reward          :  32807.487                                                  
Min. Orders          :     13.000                                                  
Avg. Orders          :     32.700                                                  
Max. Orders          :     44.000                                                  
--------------------------------------------------------------------------------   
Window size: 10, entropy: 0.02407149836842502, learning rate: 0.006777778572722783 
 96%|█████████▌| 24/25 [1:23:28<04:55, 295.47s/trial, best loss: 34440.45898532432]

  0%|          | 0/10 [00:00<?, ?it/s]
Episode: 0, Avg. Reward: -1810.299, # of orders: 2.000:   0%|          | 0/10 [00:02<?, ?it/s]
Episode: 0, Avg. Reward: -1810.299, # of orders: 2.000:  10%|#         | 1/10 [00:02<00:20,  2.30s/it]
Episode: 1, Avg. Reward: -905.150, # of orders: 1.000:  20%|##        | 2/10 [00:04<00:18,  2.30s/it] 
Episode: 1, Avg. Reward: -905.150, # of orders: 1.000:  30%|###       | 3/10 [00:04<00:09,  1.42s/it]
Episode: 2, Avg. Reward: -687.865, # of orders: 1.000:  40%|####      | 4/10 [00:06<00:08,  1.42s/it]
Episode: 2, Avg. Reward: -687.865, # of orders: 1.000:  50%|#####     | 5/10 [00:06<00:06,  1.22s/it]
Episode: 3, Avg. Reward: -620.929, # of orders: 1.000:  50%|#####     | 5/10 [00:08<00:06,  1.22s/it]
Episode: 3, Avg. Reward: -620.929, # of orders: 1.000:  60%|######    | 6/10 [00:08<00:05,  1.44s/it]
Episode: 4, Avg. Reward: -533.348, # of orders: 1.000:  70%|#######   | 7/10 [00:10<00:04,  1.44s/it]
Episode: 4, Avg. Reward: -533.348, # of orders: 

Min. Reward          :  -1810.299                                                  
Avg. Reward          :    -98.556                                                  
Max. Reward          :   3243.850                                                  
Min. Orders          :      0.000                                                  
Avg. Orders          :      0.900                                                  
Max. Orders          :      2.000                                                  
100%|██████████| 25/25 [1:29:09<00:00, 213.96s/trial, best loss: 98.55572018161766]
Best parameters: {'ent_coef': 0.02407149836842502, 'learning_rate': 0.006777778572722783, 'window_size': 0}


In [ ]:
# load in the saved trials data
# trials = pickle.load(open('hyperopt/trials.pkl', 'rb'))
# trials_4_19 = pickle.load(open('hyperopt/trials_4_19.pkl', 'rb'))
# trials_all = trials.extend(trials_4_19)
# trials_all

,loss,status,eval_time,parameters
0,NaN,fail,1.713532e+09,NaN
0,NaN,fail,1.713532e+09,NaN
0,108189.442904,ok,1.713533e+09,NaN
0,112597.585936,ok,1.713533e+09,NaN
0,NaN,fail,1.713533e+09,NaN
...,...,...,...,...
0,NaN,fail,1.713562e+09,NaN
0,NaN,fail,1.713562e+09,NaN
0,192103.142642,ok,1.713563e+09,NaN
0,189639.526575,ok,1.713563e+09,NaN


In [ ]:
results_df = pd.DataFrame()
for result in trials.results:
    result['window_size'] = result['parameters']['window_size']
    result['learning_rate'] = result['parameters']['learning_rate']
    result['ent_coef'] = result['parameters']['ent_coef']
    # del result['parameters']
    new_row = pd.DataFrame(result)
    results_df = pd.concat([results_df, new_row], axis=0)
results_df

In [ ]:
# visualize the parameters that cause failures in the objective function

# create a graph that has learning rate on the x-axis and ent_coef on the y-axis, then the color of the points is whether the status is ok or fail, green for ok and red for fail
fig, ax = plt.subplots()
scatter = ax.scatter(results_df['learning_rate'], results_df['ent_coef'], c=results_df['status'].apply(lambda x: 'green' if x == 'ok' else 'red'))
ax.set_xlabel('Learning Rate')
ax.set_ylabel('Entropy Coefficient')
ax.set_title('Hyperparameter Optimization')
plt.legend(handles=scatter.legend_elements()[0], labels=['OK', 'Fail'])
plt.show()


### Plot Results

In [17]:
# # create a dataframe of the rewards
# rewards_df = pd.DataFrame({'rewards': rewards})
# # plot the rewards
# plt.figure(figsize=(10, 5))
# sns.lineplot(data=rewards_df)
# plt.title('Rewards')
# plt.xlabel('Episode')
# plt.ylabel('Reward')
# plt.legend()
# plt.show()


In [18]:
# # import the models from /models folder
# import os
# import glob
# # get the list of models
# model_list = glob.glob('models_4_17_24/*.pkl')
# # separate the strings of each model name on _ and get the last element of the string if the string of the model doesn't include 'initial' or 'updated'
# model_list_episode_nbr = [model.split('_')[-1] for model in model_list if 'initial' not in model and 'updated' not in model]
# model_list_episode_nbr = [int(model_name.split('.')[0]) for model_name in model_list_episode_nbr]
# max_episode = max(model_list_episode_nbr)
# # test the last set of 10 episodes
# init_episode = ((int(max_episode)/10) - 10)*10
# # print(max_episode, init_episode)
# models = []
# # test the last set of 10 episodes from init_episode to max_episode
# for nbr in range(int(init_episode), int(max_episode)+10, 10):
#     # set up the appropriate time_points for each of the models in the list
#     env_train.time_points = list(symbols[1]['EURUSD'].iloc[-int(training_length):-(int(testing_length)-int(nbr)), :].index)# make this -nbr not +nbr next time
#     obs_train, info_train = env_train.reset(seed=2024)
#     # find the model name that contains the nbr
#     model_name = [model for model in model_list if str(nbr) in model][0]
#     print(model_name)
#     # load the models into a list
#     models.append(PPO.load(model_name, env=env_train))

In [19]:
# sim_testing = gym_mtsim.MtSimulator(
#     unit='USD',
#     balance=200000.,
#     leverage=100.,
#     stop_out_level=0.2,
#     hedge=True,
#     symbols_filename=FOREX_DATA_PATH
# )

# env_testing = MtEnv(
#     original_simulator=sim_testing,
#     trading_symbols=['EURUSD'],
#     window_size = window_size_param,
#     time_points=list(testing_index_slice),
#     hold_threshold=0.1,
#     close_threshold=0.1,
#     fee=lambda symbol: {
#         # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
#         'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
#         # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
#     }[symbol],
#     symbol_max_orders=2,
#     multiprocessing_processes=2
# )

In [20]:
# model_ppo = PPO.load(f'models_4_17_24\model_25K_5.pkl', env=env_train)

# obs_test, info_test = env_testing.reset(seed=2024)
# done_test = False
# while not done_test:
#     action, _states = model_ppo.predict(obs_test)
#     obs_test, reward_test, terminated_test, truncated_test, info_test = env_testing.step(action)
#     done_test = terminated_test or truncated_test
#     # total_reward += reward_test
#     if done_test:
#         break
# try:
#     order_len = len(env_testing.render()['orders'])
# except:
#     order_len = 0
# # print(f"Episode: {episode}, Reward: {total_reward:.3f}, # orders: {order_len}")


In [21]:
# # if model_dict is still a thing
# for timestep in timesteps_models_dict.keys():
#     models_dict = timesteps_models_dict[timestep]

#     for nbr in range(0, 10):
#         msg = f"{'-'*8} Testing Model {nbr} with {timestep} training timesteps {'-'*8}"
#         print(f"""{msg}\n{'-'*len(msg)}""")
#         reward_across_episodes = []
#         rewards_dict = {}
#         model_results_dict = {}
#         for episode in range(0, 10):   
#             total_reward = 0
#             done_test = False
#             model_ppo = models_dict[f'model_{nbr}']

#             obs_test, info_test = env_train.reset(seed=2024)
#             while not done_test:
#                 action, _states = model_ppo.predict(obs_test)
#                 obs_test, reward_test, terminated_test, truncated_test, info_test = env_train.step(action)
#                 done_test = terminated_test or truncated_test
#                 total_reward += reward_test
#                 if done_test:
#                     break
#             reward_across_episodes.append(total_reward)
#             try:
#                 order_len = len(env_train.render()['orders'])
#             except:
#                 order_len = 0
#             print(f"Episode: {episode}, Reward: {total_reward:.3f}, # orders: {order_len}")
#         print_stats(reward_across_episodes)
#         model_results_dict[f'model_{nbr}_{timestep}'] = reward_across_episodes
# model_results_df = pd.DataFrame(model_results_dict)

In [22]:
# # plot the rewards for each model over episodes
# plt.figure(figsize=(10, 5))
# sns.lineplot(data=model_results_df)
# plt.title('Rewards')
# plt.xlabel('Episode')
# plt.ylabel('Reward')
# plt.legend()
# plt.show()

In [23]:
# # if model_dict is still a thing
# for timestep in ['25K']:
#     for nbr in tqdm(range(1, 10)):
#         msg = f"{'-'*8} Testing Model {nbr} with {timestep} training timesteps {'-'*8}"
#         print(f"""{msg}\n{'-'*len(msg)}""")
#         reward_across_episodes = []
#         number_of_orders_across_episodes = []
#         rewards_dict = {}
#         model_results_dict = {}
#         for episode in tqdm(range(0, 10)):   
#             total_reward = 0
#             done_test = False
#             model_ppo = PPO.load(f'models_4_17_24\model_{timestep}_{nbr}.pkl', env=env_train)

#             obs_test, info_test = env_testing.reset(seed=2024)
#             while not done_test:
#                 action, _states = model_ppo.predict(obs_test)
#                 obs_test, reward_test, terminated_test, truncated_test, info_test = env_testing.step(action)
#                 done_test = terminated_test or truncated_test
#                 total_reward += reward_test
#                 if done_test:
#                     break
#             reward_across_episodes.append(total_reward)
#             try:
#                 order_len = len(env_testing.render()['orders'])
#             except:
#                 order_len = 0
#             number_of_orders_across_episodes.append(order_len)
#             print(f"Episode: {episode}, Reward: {total_reward:.3f}, # orders: {order_len}")
#         print_stats(reward_across_episodes, 'Reward')
#         print_stats(number_of_orders_across_episodes, 'Orders')
#         model_results_dict[f'model_{nbr}_{timestep}'] = reward_across_episodes
# model_results_df = pd.DataFrame(model_results_dict)

In [24]:
# # plot the rewards for each model over episodes
# plt.figure(figsize=(10, 5))
# sns.lineplot(data=model_results_df)
# plt.title('Rewards')
# plt.xlabel('Episode')
# plt.ylabel('Reward')
# plt.legend()
# plt.show()

In [25]:
# # if the cluster has been restarted
# for nbr, time_points_idx in zip(range(0, 10), range(0, 500, 50)):
#     # model_ppo.learn(total_timesteps=25000, callback=ProgressBarCallback(100))
#     env_train = MtEnv(
#         original_simulator=sim_train,
#         trading_symbols=['EURUSD'],
#         window_size = window_size_param,
#         time_points=list(symbols[1]['EURUSD'].iloc[-int(training_length):-(int(testing_length)-int(time_points_idx)), :].index),
#         hold_threshold=0.5,
#         close_threshold=0.5,
#         fee=lambda symbol: {
#             # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
#             'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
#             # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
#         }[symbol],
#         symbol_max_orders=2,
#         multiprocessing_processes=2
#     )
#     # obs_train, info_train = env_train.reset(seed=2024)
#     total_reward = 0
#     done_test = False
#     model_ppo = PPO.load(f'models\model_{nbr}.pkl', env=env_train)

#     env_testing = MtEnv(
#         original_simulator=sim_testing,
#         trading_symbols=['EURUSD'],
#         window_size = window_size_param,
#         # time_points=list(testing_index_slice),
#         hold_threshold=0.5,
#         close_threshold=0.5,
#         fee=lambda symbol: {
#             # 'GBPCAD': max(0., np.random.normal(0.0007, 0.00005)),
#             'EURUSD': max(0., np.random.normal(0.0001, 0.00003))
#             # 'USDJPY': max(0., np.random.normal(0.02, 0.003)),
#         }[symbol],
#         symbol_max_orders=2,
#         multiprocessing_processes=2
#     )
#     obs_test, info_test = env_testing.reset(seed=2024)
#     while not done_test:
#         action, _states = model_ppo.predict(obs_test)
#         obs_test, reward_test, terminated_test, truncated_test, info_test = env_testing.step(action)
#         done_test = terminated_test or truncated_test
#         total_reward += reward_test
#         if done_test:
#             break
#     state = env_testing.render()

#     print(
#         f"balance: {state['balance']}, equity: {state['equity']}, margin: {state['margin']}\n"
#         f"free_margin: {state['free_margin']}, margin_level: {state['margin_level']}\n"

#     )
#     # print(state['orders'].Profit.sum())
#     if len(state['orders']) > 0:
#         print(state['orders'].Profit.sum())

In [26]:
# max_episode = 120
# for model_nbr in range(0, int(max_episode)+10, 10):
#     print(f'Model: {model_nbr}')
#     over_episodes_rewards = []
#     over_episodes_balance = []
#     obs_training, info_training = env_train.reset(seed=2024)
#     for episode in range(0, 10):
#         obs_test, info_test = env_testing.reset(seed=2024)
        
#         # model_ppo.learn(total_timesteps=25000, callback=ProgressBarCallback(100))

#         total_reward = 0
#         done_test = False
#         env_train.time_points = list(symbols[1]['EURUSD'].iloc[-int(training_length):-(int(testing_length)-int(model_nbr)), :].index)
#         obs_training, info_training = env_train.reset(seed=2024)
#         model_ppo = PPO.load(f'models\model_{model_nbr}.pkl', env=env_train)

#         while not done_test:
#             action, _states = model_ppo.predict(obs_test)
#             obs_test, reward_test, terminated_test, truncated_test, info_test = env_testing.step(action)
#             done_test = terminated_test or truncated_test

#             total_reward += reward_test
#             if done_test:
#                 break
#         over_episodes_balance.append(info_test['balance'])
#         over_episodes_rewards.append(total_reward)
#         print(f'Episode: {episode}, Reward: {total_reward:.3f}, Balance: {info_test["balance"]:.3f}')
#     print_stats(over_episodes_rewards)
#     print_stats(over_episodes_balance)

In [27]:
# data = pd.DataFrame(plot_data)

# sns.set_style('whitegrid')
# plt.figure(figsize=(8, 6))

# for key in plot_data:
#     if key == 'x':
#         continue
#     label = plot_settings[key]['label']
#     line = plt.plot('x', key, data=data, linewidth=1, label=label)

# plt.xlabel('episode')
# plt.ylabel('reward')
# plt.title('Random vs. SB3 Agents')
# plt.legend()
# plt.show()